# Import BRAILS Importer

In [1]:
from brails.utils import Importer

# Define Location Specific Parameters

In [2]:
LOCATION_NAME = 'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

# Create and Importer object to Pull In Required BRAILS Modules

In [3]:
importer = Importer()

# Define the Region Object for the Region of Interest

In [4]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

# Get Raw NSI Data for the Defined Region

In [5]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data_given_boundary(region_boundary_object, 'ft')


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Getting National Structure Inventory (NSI) building data for the entered location input...
Found a total of 3558 building points in NSI that are within the entered region of interest


# Get FEMA USA Footprints Data for the Defined Region

In [6]:
scraper_class = importer.get_class('MS_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States


Processing quadkeys: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:16<00:00, 68.44s/it]



Found a total of 3518 building footprints in Fort Myers Beach


# Create a Baseline Inventory by Merging NSI Raw Data and USA Structures Footprints

In [14]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)


Getting National Structure Inventory (NSI) building data for the entered location input...
Found a total of 3140 building points in NSI that match the footprint data.


# Fill Gaps in the Baseline Inventory by Imputing Missing Values

In [16]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

Missing percentages among 3518 assets
buildingheight: 20.15%
fparea: 10.74%
numstories: 10.74%
erabuilt: 10.74%
repaircost: 10.74%
constype: 10.74%
occupancy: 10.74%
splitlevel: 10.74%
basement: 28.40%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.02 mins


# Predict Roof Type Information from Aerial Imagery

In [18]:
google_satellite_class = importer.get_class('GoogleSatellite')
google_satellite = google_satellite_class()
images_aerial = google_satellite.get_images(imputed_inventory,
                                            'tmp/satellite/')

roof_shape_classifier_class = importer.get_class('RoofShapeClassifier')
roof_shape_classifier = roof_shape_classifier_class()
predictions = roof_shape_classifier.predict(images_aerial)

INFO:brails.scrapers.google_satellite.google_satellite:Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/inference/tmp/satellite
Obtaining satellite imagery: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3518/3518 [00:48<00:00, 72.88it/s]



Default roof classifier model in tmp/models/roofTypeClassifier_v1.pth loaded


# Getting Design Wind Speed, Flood Zone and Roof Type Information in Created Inventory

In [19]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({'DesignWindSpeed': 130,
                      'FloodZone': 'AE',
                      'RoofShape': predictions[key]})

# Run HAZUS Rulesets to Infer R2D-Required Data

In [21]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingMaterial',
                                        'numstories': 'NumberOfStories'})
hurricaneInferer = importer.get_class("HazusHurricaneInferer")
inferer = hurricaneInferer(
    input_inventory=imputed_inventory, clean_features=False)
hazus_inferred_inventory = inferer.infer()

Done inference. It took 0.01 mins


# Re-run Imputation to Fill Values That Cannot be Inferred by HAZUS Rulesets

In [12]:
imputer = knn_imputer_class(hazus_inferred_inventory, 
                            n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inventory_final = imputer.impute()

Missing percentages among 3518 assets
lon: 10.74%
lat: 10.74%
fd_id: 10.74%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.00 mins


# Write the Created Inventory in a GeoJSON File

In [13]:
_ = hazus_inventory_final.write_to_geojson(
    output_file=INVENTORY_OUTPUT)

In [22]:
_ = hazus_inferred_inventory.write_to_geojson(
    output_file='FortMyers_AfterInferer(step4).geojson')